池化层
- 积对位置敏感。例如检查垂直边缘，可以用[1. -1]这样1x2的核，检测出垂直1的边缘。但是1像素移位就会导致0输出。因为实际拍照的时候边缘都可能发生变化，希望稍微抖动还是能保持边缘。
- 需要一定程度的平移不变性：照明，物体位置，比例，外观等等因图像而异。

二维最大池化
- 返回滑动窗口中的最大值：这里不是有核，做点积再加起来了，而是取窗口中的最大值。
- 垂直边缘检测，卷积输出后，再做2x2最大池化，允许左边的1像素移位。这样就是左边两列1.

平均池化层
- 最大池化层：每个窗口中最强的模式信号
- 平均池化层：将最大池化层中的最大操作替换为平均

总结
- 池化层返回窗口中最大或平均值
- 缓解卷积层对位置的敏感性，通常池化层作用在卷积层之后
- 同样有窗口大小、填充、和步幅作为超参数
- 输入通道数等于输出通道数，没有输出通道数作为超参数，所以池化层不会把模型变大

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install d2l==0.17.6

In [2]:
import os
path = '/content/drive/MyDrive'
os.chdir(path)

!source venv_d2l/bin/activate

path = '/content/drive/MyDrive/d2l-zh'
os.chdir(path)

In [3]:
# 实现池化层的正向传播
import torch
from torch import nn
from d2l import torch as d2l

def pool2d(X, pool_size, mode='max'):
    p_h, p_w = pool_size
    Y = torch.zeros((X.shape[0] - p_h + 1, X.shape[1] - p_w + 1))
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            if mode == 'max':
                Y[i, j] = X[i:i + p_h, j:j + p_w].max()
            elif mode == 'avg':
                Y[i, j] = X[i:i + p_h, j:j + p_w].mean()
    return Y

In [4]:
# 验证二维最大池化层的输出
X = torch.tensor([[0.0, 1.0, 2.0], [3.0, 4.0, 5.0], [6.0, 7.0, 8.0]])
pool2d(X, (2, 2))

tensor([[4., 5.],
        [7., 8.]])

In [5]:
# 验证平均池化层
pool2d(X, (2, 2), 'avg')

tensor([[2., 3.],
        [5., 6.]])

In [6]:
# 填充和步幅
# 批量大小和通道数目前唯一
X = torch.arange(16, dtype=torch.float32).reshape((1, 1, 4, 4))
X

tensor([[[[ 0.,  1.,  2.,  3.],
          [ 4.,  5.,  6.,  7.],
          [ 8.,  9., 10., 11.],
          [12., 13., 14., 15.]]]])

In [7]:
# 深度学习框架中的步幅与池化窗口的大小相同，意味着下一个窗口跟他是没有重叠的
# 这里3就是3x3的窗口
pool2d = nn.MaxPool2d(3)
pool2d(X)

tensor([[[[10.]]]])

In [8]:
# 填充和步幅可以手动设定
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

tensor([[[[ 5.,  7.],
          [13., 15.]]]])

In [9]:
# 设定一共任意大小的矩形池化窗口，并分布设定填充和步幅的高度和宽度
pool2d = nn.MaxPool2d((2, 3), padding=(1, 1), stride=(2, 3))
pool2d(X)

tensor([[[[ 1.,  3.],
          [ 9., 11.],
          [13., 15.]]]])

In [2]:
# 池化层在每个输入通道上单独运算
X = torch.cat((X, X + 1), 1)
X

In [ ]:
pool2d = nn.MaxPool2d(3, padding=1, stride=2)
pool2d(X)

池化时窗口有重叠与没有重叠可能没有什么区别。现在池化层用的越来越少了。
Python调用速度不是很高，创造矩阵直接创造就好了，尽量不要调用python的函数。如果不知道矩阵的尺寸，可以用python的list of list append，再转成tensor。

为什么池化层用得越来越少了？
- 现在通常用卷积层+stride来减少计算。
- 数据会做大量增强，使得数据本身会发生很多变化，淡化了池化层的作用。
- 最后的最后可以用一个池化层变成一个单数字。



池化层（Pooling Layer）通常在卷积神经网络（CNN）中使用，其主要作用是对输入特征图（feature map）进行下采样或降维，从而减少数据的空间尺寸，提高计算效率，减少模型的参数数量，从而控制过拟合。然而，池化层的作用主要是特征提取和变换的一部分，它通过提取局部区域内的统计特征（如最大值或平均值）来实现这些功能。

尽管池化层可以帮助减少过拟合，但它自身并不算是一种正则化技术。传统的正则化技术如L1正则化、L2正则化、Dropout等，主要是通过在训练过程中引入额外的约束或修改损失函数来直接控制模型的复杂度。池化层更准确地说是一种结构化的方法，通过改变网络的结构来影响模型的性能和复杂度。

因此，尽管池化可以间接帮助控制模型复杂度和过拟合，但它本身并不属于正则化技术的一部分。